#Trying agian by changing the learning rate from 2e-5 to 3e-5

# BERT Fine-tuning for GoEmotions (continued)

This notebook continues the fine-tuning pipeline. It initializes the tokenizer and model, wraps the tokenized data in a PyTorch Dataset, configures Trainer-based training, computes multi-label metrics, saves the model, and runs a few sample inferences.

Assumptions: the preprocessed CSV files are available in the workspace and contain a `text` column and one-hot label columns (one column per emotion).

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [2]:
!pip install transformers datasets accelerate tokenizers
!pip install scikit-learn pandas jupyterlab notebook ipywidgets
!pip install nbconvert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.8 MB/s eta 0:00:00


In [3]:
# 1. Imports and basic setup
import os
import glob
import random
from dataclasses import dataclass
from typing import List, Dict

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed,
)

# Reproducibility
SEED = 42
set_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)

Using device: cuda


In [4]:
# 2. Load preprocessed CSVs and infer label columns
data_files = sorted(glob.glob('Preprocessed_goemotions_*.csv'))
if not data_files:
    # fallback to single file if pattern doesn't match
    data_files = [
        'Preprocessed_goemotions_1.csv'
    ]
print('Found data files:', data_files)

# Read and concatenate (be careful with memory for large datasets)
dfs = [pd.read_csv(f) for f in data_files]
df = pd.concat(dfs, ignore_index=True)
print('Total examples:', len(df))

# Expect a `text` column and multiple binary label columns. Infer label columns as all columns except 'text'.
assert 'text' in df.columns, 'Expected a text column in CSV'
label_cols = [c for c in df.columns if c != 'text']
num_labels = len(label_cols)
print('Detected label columns (%d):' % num_labels, label_cols)

Found data files: ['Preprocessed_goemotions_1.csv', 'Preprocessed_goemotions_2.csv', 'Preprocessed_goemotions_3.csv']
Total examples: 183356
Detected label columns (9): ['Joyful', 'Affectionate', 'Positive_Outlook', 'Anger_Frustration', 'Sadness_Disappointment', 'Fear_Anxiety', 'Surprise_Confusion', 'Desire', 'Neutral']


## Tokenizer and Model Initialization
We'll use `bert-base-uncased` and configure the model for multi-label classification.

In [5]:
MODEL_NAME = 'bert-base-uncased'
MAX_LENGTH = 128

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# Config -> ensure problem_type set for multi-label if appropriate
config = AutoConfig.from_pretrained(MODEL_NAME, num_labels=num_labels)
# If this is multi-label (one-hot columns), set the problem_type so HF uses BCEWithLogitsLoss
config.problem_type = 'multi_label_classification'

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config)
model.to(device)
print('Model loaded. num_labels =', model.config.num_labels, 'problem_type =', model.config.problem_type)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded. num_labels = 9 problem_type = multi_label_classification


## Dataset Wrapper
Create a PyTorch Dataset that returns tokenized inputs and float labels (for BCE loss).

In [6]:
class GoEmotionsDataset(Dataset):
    def __init__(self, texts: List[str], labels: List[List[float]], tokenizer, max_length: int = 128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = np.array(self.labels[idx], dtype=np.float32)
        enc = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors=None,
        )
        item = {
            'input_ids': torch.tensor(enc['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(enc['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.float),
        }
        return item

# Prepare train/val split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=SEED, shuffle=True)
print('Train / Val sizes:', len(train_df), len(val_df))

train_texts = train_df['text'].fillna('').tolist()
val_texts = val_df['text'].fillna('').tolist()
train_labels = train_df[label_cols].values.tolist()
val_labels = val_df[label_cols].values.tolist()

train_dataset = GoEmotionsDataset(train_texts, train_labels, tokenizer, max_length=MAX_LENGTH)
val_dataset = GoEmotionsDataset(val_texts, val_labels, tokenizer, max_length=MAX_LENGTH)

print('Datasets ready. Example item keys:', list(train_dataset[0].keys()))

Train / Val sizes: 165020 18336
Datasets ready. Example item keys: ['input_ids', 'attention_mask', 'labels']


## Training Configuration (Hugging Face Trainer)
Set hyperparameters and enable device-aware training.

In [7]:
# Hyperparameters (tweak as needed)
BATCH_SIZE = 16
LEARNING_RATE = 5e-5
NUM_EPOCHS = 3
WEIGHT_DECAY = 0.01
LOGGING_STEPS = 100
OUTPUT_DIR = './bert_finetuned'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Enable fp16 if CUDA available
fp16 = True if device == 'cuda' else False

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=WEIGHT_DECAY,
    logging_steps=LOGGING_STEPS,
    load_best_model_at_end=False,
    fp16=fp16,
    report_to='none',
)

print('TrainingArguments configured. fp16=', fp16)

TrainingArguments configured. fp16= True


## Metrics & compute_metrics function
Define metrics for multi-label evaluation. Predictions are logits; apply sigmoid + thresholding.

In [8]:
def compute_metrics(eval_pred):
    # eval_pred.predictions: logits, eval_pred.label_ids: true labels
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    if isinstance(logits, tuple):
        logits = logits[0]
    probs = torch.sigmoid(torch.from_numpy(logits)).numpy()
    preds = (probs >= 0.28).astype(int)
    # Subset accuracy (exact match)
    try:
        acc = accuracy_score(labels, preds)
    except Exception:
        acc = 0.0
    # Macro and micro averages for precision/recall/f1
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds, average='macro', zero_division=0)
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(labels, preds, average='micro', zero_division=0)
    return {
        'accuracy': acc,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'precision_micro': precision_micro,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
    }

# Prepare Trainer - note: our dataset returns dictionaries -> Trainer will collate them properly
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
print('Trainer ready')

Trainer ready


/tmp/ipython-input-1421156522.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Training Execution
Run training and monitor progress. This cell will execute the training loop managed by the Trainer.

In [9]:
# Train
train_result = trainer.train()
trainer.save_model(OUTPUT_DIR)  # saves the tokenizer too via Trainer.save_model
metrics = train_result.metrics
print('Training finished. Summary metrics:')
print(metrics)

# Save training metrics to disk
with open(os.path.join(OUTPUT_DIR, 'train_metrics.json'), 'w') as f:
    import json
    json.dump(metrics, f, indent=2)

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro,Precision Micro,Recall Micro,F1 Micro
1,0.224700,0.221834,0.400960,0.482875,0.516599,0.486689,0.497462,0.646760,0.562371
2,0.209400,0.219085,0.420321,0.474270,0.524044,0.490689,0.507065,0.638034,0.565060
3,0.184400,0.229331,0.422720,0.446562,0.537666,0.486163,0.497074,0.630017,0.555705


Training finished. Summary metrics:
{'train_runtime': 3820.4333, 'train_samples_per_second': 129.582, 'train_steps_per_second': 8.099, 'total_flos': 3.256598641595904e+16, 'train_loss': 0.20853760413366562, 'epoch': 3.0}


## Evaluation & Detailed Reports
Run prediction on the validation set, compute a classification report and confusion matrices per label.

In [10]:
# Predict on validation set
pred_output = trainer.predict(val_dataset)
logits = pred_output.predictions
if isinstance(logits, tuple):
    logits = logits[0]
probs = torch.sigmoid(torch.from_numpy(logits)).numpy()
preds = (probs >= 0.28).astype(int)
true = pred_output.label_ids

# Overall metrics (same as compute_metrics)
overall = compute_metrics(pred_output)
print('Overall evaluation metrics:')
print(overall)

# Per-label classification reports
for i, col in enumerate(label_cols):
    print('Label:', col)
    try:
        rep = classification_report(true[:, i], preds[:, i], zero_division=0)
        print(rep)
    except Exception as e:
        print('Failed to compute report for', col, e)

# Per-label confusion matrices (printed as small tables)
print('Per-label confusion matrices:')
for i, col in enumerate(label_cols):
    cm = confusion_matrix(true[:, i], preds[:, i])
    print(col, 'confusion_matrix:', cm)

Overall evaluation metrics:
{'accuracy': 0.42272033158813266, 'precision_macro': 0.4465617678211901, 'recall_macro': 0.5376663695501555, 'f1_macro': 0.48616263528303627, 'precision_micro': 0.49707401032702236, 'recall_micro': 0.6300174520069808, 'f1_micro': 0.555705214546854}
Label: Joyful
              precision    recall  f1-score   support

         0.0       0.97      0.95      0.96     16906
         1.0       0.49      0.61      0.54      1430

    accuracy                           0.92     18336
   macro avg       0.73      0.78      0.75     18336
weighted avg       0.93      0.92      0.92     18336

Label: Affectionate
              precision    recall  f1-score   support

         0.0       0.92      0.84      0.88     14266
         1.0       0.57      0.75      0.65      4070

    accuracy                           0.82     18336
   macro avg       0.75      0.80      0.76     18336
weighted avg       0.84      0.82      0.83     18336

Label: Positive_Outlook
           

## Save Model & Tokenizer
The trained model and tokenizer are saved to `./bert_finetuned`.

In [11]:
# Confirm saved files
print('Saved files in output dir:')
print(''.join(sorted(os.listdir(OUTPUT_DIR))))

Saved files in output dir:
checkpoint-10314checkpoint-20628checkpoint-30942config.jsonmodel.safetensorsspecial_tokens_map.jsontokenizer.jsontokenizer_config.jsontrain_metrics.jsontraining_args.binvocab.txt


## Sample Inference
Run the fine-tuned model on a few validation samples to print predicted labels and probabilities.

In [12]:
# Load model/tokenizer from output dir for inference (ensures reproducibility)
inference_tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR, use_fast=True)
inference_model = AutoModelForSequenceClassification.from_pretrained(OUTPUT_DIR)
inference_model.to(device)
inference_model.eval()

def predict_texts(texts: List[str], threshold: float = 0.5) -> List[Dict]:
    enc = inference_tokenizer(texts, padding=True, truncation=True, max_length=MAX_LENGTH, return_tensors='pt')
    for k in enc:
        enc[k] = enc[k].to(device)
    with torch.no_grad():
        out = inference_model(**enc)
        logits = out.logits
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= threshold).astype(int)
    results = []
    for p, pr in zip(preds, probs):
        label_inds = [label_cols[i] for i, v in enumerate(p) if v == 1]
        scores = {label_cols[i]: float(pr[i]) for i in range(len(label_cols))}
        results.append({'predicted_labels': label_inds, 'scores': scores})
    return results

sample_texts = val_texts[:5]
preds = predict_texts(sample_texts)
for t, p in zip(sample_texts, preds):
    print('TEXT:', t)
    print('PREDICTED:', p['predicted_labels'])
    # print top 3 scores
    top3 = sorted(p['scores'].items(), key=lambda x: x[1], reverse=True)[:3]
    print('TOP SCORES:', top3)
    print('-' * 60)

TEXT: She needs a raise.
PREDICTED: ['Neutral']
TOP SCORES: [('Neutral', 0.8532434105873108), ('Affectionate', 0.08079467713832855), ('Desire', 0.06232110410928726)]
------------------------------------------------------------
TEXT: I just want tk be a random scumbag
PREDICTED: []
TOP SCORES: [('Desire', 0.46908038854599), ('Neutral', 0.44498661160469055), ('Anger_Frustration', 0.04726702719926834)]
------------------------------------------------------------
TEXT: Well, this is the first time I've seen that in over three years.
PREDICTED: []
TOP SCORES: [('Surprise_Confusion', 0.44924911856651306), ('Joyful', 0.22332318127155304), ('Neutral', 0.21328699588775635)]
------------------------------------------------------------
TEXT: Welcome to racial integration, where your civility standards are racist and you better shut up or else.
PREDICTED: []
TOP SCORES: [('Anger_Frustration', 0.425889253616333), ('Affectionate', 0.37202489376068115), ('Neutral', 0.14205297827720642)]
-------------